### Data Preprocessing

In [1]:
fldr="drive/My Drive/amazon_food_reviews"

In [2]:
import os
files=os.listdir(fldr)


In [3]:
print(files)

['Reviews.csv', 'database.sqlite', 'hashes.txt', 'summaries.npy', 'reviews.npy']


In [4]:
import pandas as pd
df=pd.read_csv(fldr+'/Reviews.csv')

In [5]:
print(df.head())

   Id  ...                                               Text
0   1  ...  I have bought several of the Vitality canned d...
1   2  ...  Product arrived labeled as Jumbo Salted Peanut...
2   3  ...  This is a confection that has been around a fe...
3   4  ...  If you are looking for the secret ingredient i...
4   5  ...  Great taffy at a great price.  There was a wid...

[5 rows x 10 columns]


In [6]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [7]:
df.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [8]:
df.dropna(subset=['Summary'],inplace=True)  ###Removing the ones with null Summary

In [9]:
df.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                    0
Text                       0
dtype: int64

In [10]:
df_final=df[['Text','Summary']]

In [11]:
df_final.head()

,Text,Summary
0,I have bought several of the Vitality canned d...,Good Quality Dog Food
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised
2,This is a confection that has been around a fe...,"""Delight"" says it all"
3,If you are looking for the secret ingredient i...,Cough Medicine
4,Great taffy at a great price. There was a wid...,Great taffy


In [12]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
reviews_cleaned=[]
summary_cleaned=[]

In [14]:
import re
def obtain_clean(stop_words):
  i=0
  summaries=[]
  reviews=[]
  while i<len(df_final):
    sen_sum=""
    sen_rev=""
    
      #print(extract)
    clean_review=re.sub("[^a-zA-Z0-9]"," ",df_final.iloc[i]['Text'])   ## Removing special characters
    clean_summary=re.sub("[^a-zA-Z0-9]"," ",df_final.iloc[i]['Summary'])
    #print(clean_sentence)
    obtained_rev=word_tokenize(clean_review)
    obtained_sum=word_tokenize(clean_summary) 
      
    for word in obtained_rev:
        word_final=word.lower()
        if word_final not in stop_words:
          sen_rev=sen_rev+word_final+' '
    
    for word in obtained_sum:
        word_final=word.lower()
        if word_final not in stop_words:
          sen_sum=sen_sum+word_final+' '
      #print(sen)  
    summaries.append(sen_sum)
    reviews.append(sen_rev)
    i+=1

  return summaries,reviews

In [15]:
summaries,reviews=obtain_clean(stop_words)

In [16]:
len(summaries)

568427

In [17]:
len(reviews)

568427

In [18]:
text_summary=pd.DataFrame(list(zip(reviews,summaries)),columns=['Reviews','Summaries'])

In [19]:
text_summary.head()

,Reviews,Summaries
0,bought several vitality canned dog food produc...,good quality dog food
1,product arrived labeled jumbo salted peanuts p...,advertised
2,confection around centuries light pillowy citr...,delight says
3,looking secret ingredient robitussin believe f...,cough medicine
4,great taffy great price wide assortment yummy ...,great taffy


In [20]:
text_summary.isnull().sum()

Reviews      0
Summaries    0
dtype: int64

In [21]:
text=text_summary[['Reviews']].values

In [22]:
text

array([['bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better '],
       ['product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo '],
       ['confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story c lewis lion witch wardrobe treat seduces edmund selling brother sisters witch '],
       ...,
       ['stars small give 10 15 one training session tried train dog ceaser dog treats made puppy hyper compare ingredients know little stars basic food ingredients without preservatives food coloring sweet potato flavor also make hand smell like dog food '],
       ['best treats training rewarding dog good grooming lower calories loved doggies sweet potatoes see

In [23]:
summaries=text_summary[['Summaries']].values

In [24]:
summaries

array([['good quality dog food '],
       ['advertised '],
       ['delight says '],
       ...,
       ['perfect maltipoo '],
       ['favorite training reward treat '],
       ['great honey ']], dtype=object)

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train,X_test,Y_train,Y_test=train_test_split(text_summary['Reviews'],text_summary['Summaries'],test_size=0.2)

In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [28]:
tokenizer = Tokenizer()

In [29]:
tokenizer.fit_on_texts(X_train)

In [30]:
x_train = tokenizer.texts_to_sequences(X_train) 
x_test = tokenizer.texts_to_sequences(X_test)

In [39]:
from keras.preprocessing.sequence import pad_sequences
maxlen=60
x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [40]:
tokenizer_sum = Tokenizer()
tokenizer_sum.fit_on_texts(Y_train)

In [41]:
y_train = tokenizer_sum.texts_to_sequences(Y_train) 
y_test = tokenizer_sum.texts_to_sequences(Y_test)

In [42]:
from keras.preprocessing.sequence import pad_sequences
maxlen=10
y_train = pad_sequences(y_train, padding='post', maxlen=maxlen)
y_test = pad_sequences(y_test, padding='post', maxlen=maxlen)

In [43]:
vocab_x=len(tokenizer.word_index)+1

In [44]:
vocab_y=len(tokenizer_sum.word_index)+1

### Taking Embeddings

In [45]:
file_path="drive/My Drive/glove.6B/"

In [46]:
import numpy as np

In [47]:
emb_dim=50
emb_mat= np.zeros((vocab_x,emb_dim))

In [48]:
with open(file_path+'glove.6B.50d.txt') as f:
        for line in f:
            word, *emb = line.split()
            if word in tokenizer.word_index:
              ind=tokenizer.word_index[word]
              emb_mat[ind]=np.array(emb,dtype="float32")[:emb_dim]

In [49]:
emb_dim=50
emb_mat_sum= np.zeros((vocab_y,emb_dim))

In [50]:
with open(file_path+'glove.6B.50d.txt') as f:
        for line in f:
            word, *emb = line.split()
            if word in tokenizer_sum.word_index:
              ind=tokenizer_sum.word_index[word]
              emb_mat_sum[ind]=np.array(emb,dtype="float32")[:emb_dim]

### Modelling

In [51]:
emb_mat_2= np.zeros((vocab_x,emb_dim))
emb_mat_sum_2= np.zeros((vocab_y,emb_dim))

In [53]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding,Dense, Activation, LSTM, Attention,Concatenate,Input,TimeDistributed
from tensorflow.keras.optimizers import Adam
emb_dim=50
#maxlen=100

In [ ]:
## Encoder
encoder_in=Input(shape=(50,))
encoder_emb=Embedding(input_dim=vocab_x,output_dim=emb_dim,weights=[emb_mat],input_length=60,trainable=False)(encoder_in)

encoder_lstm_1=LSTM(512,return_sequences=True,return_state=True)
output_1,state_h,state_c=encoder_lstm_1(encoder_emb)

encoder_lstm_2=LSTM(512,return_sequences=True,return_state=True)
output_2,state_h,state_c=encoder_lstm_2(output_1)

encoder_lstm_3=LSTM(512,return_sequences=True, return_state=True)
output_3,state_h,state_c=encoder_lstm_3(output_2)

encoder_lstm_4=LSTM(512,return_sequences=True,return_state=True)
encoder_output,state_h,state_c=encoder_lstm_4(output_3)


## Decoder
decoder_in=Input(shape=(50,))
decoder_emb=Embedding(input_dim=vocab_y,output_dim=emb_dim,weights=[emb_mat_sum],input_length=10,trainable=False)(decoder_in)

decoder_lstm_1=LSTM(512,return_sequences=True,return_state=True)
decoder_out_1, state_h_d,state_c_d=decoder_lstm_1(decoder_emb, initial_state=[state_h,state_c])

decoder_lstm_2=LSTM(512,return_sequences=True,return_state=True)
decoder_out_2, state_h_d,state_c_d=decoder_lstm_2(decoder_out_1)

decoder_lstm_3=LSTM(512,return_sequences=True,return_state=True)
decoder_output, state_h_d,state_c_d=decoder_lstm_3(decoder_out_2)

attention_out=Attention()([encoder_output,decoder_output])

concat_out=Concatenate(axis=-1)([decoder_output,attention_out])

decoder_softmax=Dense(vocab_y,activation='softmax')
outputs=decoder_softmax(concat_out)


model=Model([encoder_in,decoder_in],outputs)







In [104]:
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 100, 100)     10871500    input_12[0][0]                   
__________________________________________________________________________________________________
lstm_35 (LSTM)                  [(None, 100, 512), ( 1255424     embedding_11[0][0]               
__________________________________________________________________________________________________
lstm_36 (LSTM)                  [(None, 100, 512), ( 2099200     lstm_35[0][0]                    
_______________________________________________________________________________________

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


In [ ]:
history = model.fit([x_train,y_train[:,:-1]], y_train.reshape(y_train[0],y_train[1],1)[:,1,:],epochs=50,verbose=True,batch_size=256)